# Preprossesering med Python

Før vi kan begynne å jobbe med dataene må vi ta en titt på dem og se at alt ser ok ut, og rense opp der det ikke gjøre det. Vi må sjekke at alt har blitt lastet inn skikkelig, at alle kolonner er av riktig type og se om det er noen kolonner som mangler for mye informasjon til at vi får brukt dem. Denne første fasen kalles preprossesering og er veldig viktig for å bli kjent med datasettet og få god datakvalitet for de påfølgende analysene. 

Vi starter med å importere pakkene vi trenger og å laste inn dataene over ledige stillinger fra 2002 til 2017:

In [21]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Det er lurt å se på filene før vi importerer dem. Da ser vi at '\*' brukes som tegn for manglende data, og kan 'si' det videre til read_csv-funksjonen. Hos meg så blir csv'er lagret som semikolonseparerte filer i stedet for kommaseparerte. Hvis dine er kommaseparerte er det bare å fjerne sep-argumentet i koden under, ettersom funksjonen som default forventer en kommaseparert fil.

In [5]:
period = [2002, 2017]
jobs = pd.DataFrame()

for year in range(period[0], period[1]+1):
    file = '/Users/A153722/Documents/apne data/data/Ledigestillinger_'+str(year)+'.csv'
    yearly = pd.read_csv(file, na_values='*', sep=';')
    jobs = jobs.append(yearly, ignore_index=True)


/Users/A153722/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Når vi laster inn dataene så får vi en feilmelding - kolonne 0 og 1 har blandede datatyper. Ser vi i filene så ser det ut som begge kolonnene skal være numeriske, så det har nok sneket seg inn litt tekst innimellom tallene. Hvis vi legger til 'print(year)' i loopen over så ser vi at problemet stammer fra 2005-filen.

La oss først bare sjekke at 2015 har blitt lastet inn. Jeg legger til en kolonne med årstallene for å få oppsummeringen gruppert på år i stedet for måned.

In [13]:
#yyyymm -> yyyy
jobs['aarstall'] = jobs.statistikk_aar_mnd//100
jobs['aarstall'].value_counts(sort=False)

2002    149719
2003    129948
2004    132774
2005    157622
2006    214729
2007    255421
2008    230474
2009    167756
2010    158195
2011    167762
2012    160014
2013    143281
2014    135097
2015    135871
2016    147173
2017    176285
Name: aarstall, dtype: int64

Her ser vi at alle år er med. Men selv om kollonnene (hovedsakelig) består av tall så er ikke dette *egentlig* numeriske data - verdiene betyr ikke noe i seg selv, og rekkefølgen har ingenting å si. Det er egentlig (nominelle) kategoriske data. Vi kan la kolonnene stå som string, eller vi kan gjøre dem om til 'categorical', så Python kjenner dem igjen som det. Før vi bestemmer oss kan vi sjekke hvor mange grupper det ville blitt i så fall, og hvilken type de har nå.

In [24]:
jobs['stillingsnummer'].value_counts().size
jobs['nav_enhet_kode'].value_counts().size

jobs['stillingsnummer'].dtype
jobs['nav_enhet_kode'].dtype

2529269

756

dtype('O')

dtype('O')

Begge kolonnene er lagret som objects, som vil si string. En ting jeg har lært mens jeg undersøkte disse to kolonnene er at selv om kolonnene er lagret som objects så oppfører de seg som tall, med unntak av de fra 2015-filen. Bare se her:

In [25]:
# Random number from 2002 does not match with string-search:
sum(jobs.stillingsnummer == '2041200211000000')
#But matches with number-search:
sum(jobs.stillingsnummer == 2041200211000000)

#Random number from 2005 does match with string-search:
sum(jobs.stillingsnummer == '2041200512000000')
#But not with number-search:
sum(jobs.stillingsnummer == 2041200512000000)

0

6

4

0

Så vi må gjøre et lite triks for å ikke få overlappende kategorier:

In [26]:
jobs.iloc[:, 0:2]  = jobs.iloc[:, 0:2].astype(str)

#Number from 2002
sum(jobs.stillingsnummer == '2041200211000000')

6

Etter å ha gjort om kolonnen til string så får vi altså treff med søk på string også utenfor 2005-filen. Dette har også kosekvenser for grupperingen:

In [27]:
jobs['stillingsnummer'].value_counts().size
jobs['nav_enhet_kode'].value_counts().size

2529245

704

Det er 704 ulike nav-enheter og litt over 2.5 millioner unike stillingsnummer, nesten like mange som det er rader i dataframen vår. For stillingsnummerene gir det mest mening å beholde dem som strings, siden det er noe som en (ikke-unik) ID. Enhetskodene kan vi derimot kode om til kategorier.

In [29]:
jobs.nav_enhet_kode = jobs.nav_enhet_kode.astype('category')
jobs.nav_enhet_kode.dtypes  # check if it worked

CategoricalDtype(categories=['100', '1001', '1002', '1003', '1004', '1008', '101', '1010',
                  '1014', '1015',
                  ...
                  '928', '929', '935', '937', '938', '940', '941', '950',
                  '9999', 'aetat'],
                 ordered=False)

Nå kan vi fortsette med å undersøke dataene. Vi kan starte med å bare se på de første fem radene for å få en følelse av dataene. For å få med alle kolonnene må vi endre en instilling.

In [30]:
pd.set_option('display.max_columns', jobs.shape[1])
jobs.head()

,stillingsnummer,nav_enhet_kode,registrert_dato,sistepubl_dato,statistikk_aar_mnd,offisiell_statistikk_flagg,stilling_kilde,arbeidssted_fylkesnummer,arbeidssted_fylke,arbeidssted_kommunenummer,arbeidssted_kommune,arbeidssted_landkode,arbeidssted_land,isco_versjon,yrke_grovgruppe,yrkeskode,yrke,yrkesbetegnelse,virksomhet_organisasjonsnr,virksomhet_navn,antall_stillinger,stillingstittel,aarstall
0,101200201000001,101,08.01.2002,17.01.2002,200201,1,Annonsert i media,6.0,Buskerud,625.0,Nedre Eiker,NO,Norge,ISCO-88,Ledere (2001-2011),1223,Produksjonsdirektører innen bygge- og anleggsv...,Leder (bygg og anlegg),NaN,NaN,2,Anleggsledere/formenn,2002
1,101200201000002,101,09.01.2002,18.01.2002,200201,1,Annonsert i media,1.0,Østfold,101.0,Halden,NO,Norge,ISCO-88,"Helse, pleie og omsorg (2001-2011)",3226,"Fysioterapeuter, ergoterapeuter o.l.",Fysioterapeut,974633965.0,SYKEHUSET ØSTFOLD,1,Fysioterapeut I,2002
2,101200205000004,101,23.05.2002,27.05.2002,200205,1,Annonsert i media,3.0,Oslo,301.0,Oslo,NO,Norge,ISCO-88,Meglere og konsulenter (2001-2011),3415,Tekniske og kommersielle salgsrepresentanter,Teknisk salgsrepresentant,NaN,NaN,1,Salgskonsulent,2002
3,105200112000060,105,19.12.2001,20.12.2001,200201,1,Meldt til NAV lokalt,1.0,Østfold,104.0,Moss,NO,Norge,ISCO-88,Serviceyrker og annet arbeid (2001-2011),9132,Rengjøringspersonale i bedrifter o.l.,Renholder,NaN,NaN,1,Renholder,2002
4,105200112000061,105,19.12.2001,29.12.2001,200201,1,Annonsert i media,1.0,Østfold,104.0,Moss,NO,Norge,ISCO-88,"Helse, pleie og omsorg (2001-2011)",2224,Farmasøyter,Farmasøyt (cand.pharm.),NaN,NaN,1,Daglig leder (bestyrer) (st.nr. 9501011),2002


Vi kan også skrive ut de mest vanlige numeriske oppsummeringene, selv om det i vårt tilfelle ikke er relevant for mange av kolonnene, fordi de er kategoriske eller strings:

In [33]:
jobs.describe()

,statistikk_aar_mnd,offisiell_statistikk_flagg,arbeidssted_fylkesnummer,arbeidssted_kommunenummer,yrkeskode,virksomhet_organisasjonsnr,antall_stillinger,aarstall
count,2.662121e+06,2.662121e+06,2.628721e+06,2.628721e+06,2.662121e+06,2.132936e+06,2.662121e+06,2.662121e+06
mean,2.009425e+05,9.913723e-01,9.279508e+00,9.320196e+02,4.218345e+03,9.673685e+08,1.549483e+00,2.009365e+03
std,4.419866e+02,9.248391e-02,6.566508e+00,5.826097e+02,2.191745e+03,3.210878e+07,2.249433e+00,4.420114e+00
min,2.002010e+05,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,8.115451e+08,1.000000e+00,2.002000e+03
25%,2.006050e+05,1.000000e+00,3.000000e+00,3.010000e+02,2.411000e+03,9.731524e+08,1.000000e+00,2.006000e+03
50%,2.009040e+05,1.000000e+00,1.000000e+01,1.001000e+03,3.433000e+03,9.746246e+08,1.000000e+00,2.009000e+03
75%,2.013050e+05,1.000000e+00,1.400000e+01,1.443000e+03,5.223000e+03,9.816601e+08,2.000000e+00,2.013000e+03
max,2.017120e+05,1.000000e+00,9.900000e+01,2.435000e+03,9.629000e+03,9.996700e+08,8.000000e+02,2.017000e+03


Her ser vi for eksempel at over 99 % av statistikken er offisiell. Ellers er det noen oppsiktsvekkende tall for fylkes- og kommunenummer og yrkeskode - minimumsverdien er -1 og , som er en ugyldig verdi. Høyeste verdi for fylkesnummer er 99, som heller ikke svarer til noe fylke. La oss se litt nærmere på noen av disse radene, og sjekke hvor mange det er snakk om.

In [34]:
jobs[jobs.arbeidssted_fylkesnummer < 1].head()
jobs[jobs.arbeidssted_fylkesnummer < 1].shape

,stillingsnummer,nav_enhet_kode,registrert_dato,sistepubl_dato,statistikk_aar_mnd,offisiell_statistikk_flagg,stilling_kilde,arbeidssted_fylkesnummer,arbeidssted_fylke,arbeidssted_kommunenummer,arbeidssted_kommune,arbeidssted_landkode,arbeidssted_land,isco_versjon,yrke_grovgruppe,yrkeskode,yrke,yrkesbetegnelse,virksomhet_organisasjonsnr,virksomhet_navn,antall_stillinger,stillingstittel,aarstall
40,105200201000028,105,2002-01-08,2002-01-31,200201,1,Meldt til NAV lokalt,-1.0,ENT,-1.0,T,-1,UKJENT,ISCO-88,Reiseliv og transport (2001-2011),8323,Lastebil- og vogntogførere,Lastebil- og vogntogfører,NaN,NaN,1,Sjåfør,2002
426,105200204000067,105,2002-04-15,2002-05-01,200204,1,Reg av arb.giver på nav.no,-1.0,ENT,-1.0,T,-1,UKJENT,ISCO-88,Meglere og konsulenter (2001-2011),1239,Andre spesialdirektører,Leder (privat virksomhet),NaN,NaN,1,Daglig leder,2002
427,105200204000068,105,2002-04-15,2002-04-30,200204,1,Reg av arb.giver på nav.no,-1.0,ENT,-1.0,T,-1,UKJENT,ISCO-88,Ingeniør- og ikt-fag (2001-2011),2142,Sivilingeniører (bygg og anlegg),Sivilingeniør (bygg og anlegg),NaN,NaN,4,Sivilingeniører/ingeniører,2002
428,105200204000069,105,2002-04-15,2002-05-15,200204,1,Reg av arb.giver på nav.no,-1.0,ENT,-1.0,T,-1,UKJENT,ISCO-88,Akademiske yrker (2001-2011),2541,Sosial- og siviløkonomer,Sosialøkonom,NaN,NaN,1,Økonomer/ markedsførere / selgere,2002
429,105200204000070,105,2002-04-15,2002-05-15,200204,1,Reg av arb.giver på nav.no,-1.0,ENT,-1.0,T,-1,UKJENT,ISCO-88,Ingeniør- og ikt-fag (2001-2011),2142,Sivilingeniører (bygg og anlegg),Sivilingeniør (bygg og anlegg),NaN,NaN,10,Sivilingeniører / ingeniører,2002


(1365, 23)

Her er det mye informasjon som mangler, til og med land er ukjent. La oss sette verdier vi vet er ugyldige til NaN:

In [35]:
ranges = {'arbeidssted_fylkesnummer': range(1, 23), 'arbeidssted_kommunenummer': range(101, 2400), 'yrkeskode': range(1, 10000)}

for column in ranges:
    a = -jobs[column].isin(ranges[column])  # All values outside the range is True (- negates)
    jobs.loc[a, column] = np.nan


SyntaxError: invalid syntax (<ipython-input-35-fdf42282316c>, line 8)

Jeg vet ikke noe om yrkeskodene, men har googlet meg frem til litt info: de skal være fire- eller syvsifret og kan begynne med null, men kan ikke være bare nuller ([SSB](https://www.ssb.no/a/yrke/) og [skatteetaten](https://www.skatteetaten.no/bedrift-og-organisasjon/arbeidsgiver/a-meldingen/veiledning/arbeidsforholdet/opplysninger-om-arbeidsforholdet/yrkeskode/)). I oppsummeringen fra describe-funksjonen ser vi at det i filene kun er brukt den firesifrede koden.

Vi kan også ta en titt på hvilke typer de andre kolonnene har blitt, og om vi eventuelt bør endre på noe mer.

In [31]:
jobs.dtypes

stillingsnummer                 object
nav_enhet_kode                category
registrert_dato                 object
sistepubl_dato                  object
statistikk_aar_mnd               int64
offisiell_statistikk_flagg       int64
stilling_kilde                  object
arbeidssted_fylkesnummer       float64
arbeidssted_fylke               object
arbeidssted_kommunenummer      float64
arbeidssted_kommune             object
arbeidssted_landkode            object
arbeidssted_land                object
isco_versjon                    object
yrke_grovgruppe                 object
yrkeskode                        int64
yrke                            object
yrkesbetegnelse                 object
virksomhet_organisasjonsnr     float64
virksomhet_navn                 object
antall_stillinger                int64
stillingstittel                 object
aarstall                         int64
dtype: object

Fra utskriften over ser vi at datoene er lagret som tekst. Disse kan vi konvertere til datoformat. Pandas funksjon to_datetime virker bare på endimensjonal input, så jeg bruker apply for å kunne gjøre det på begge dato-kolonnene samtidig. Datoene blir nå printet ut i yyyy-mm-dd format.

In [32]:
jobs.iloc[:, 2:4] = jobs.iloc[:, 2:4].apply(pd.to_datetime, errors='coerce', format='%d.%m.%Y')
jobs.iloc[:, 2:4].head()

,registrert_dato,sistepubl_dato
0,2002-01-08,2002-01-17
1,2002-01-09,2002-01-18
2,2002-05-23,2002-05-27
3,2001-12-19,2001-12-20
4,2001-12-19,2001-12-29


Det er en del feilpunching i kolonnen 'sistepubl_dato' som gjør at vi får feilmeldinger hvis vi prøver å konvertere uten å bruke errors-argumentet. 'Coerce' vil si at alle felter som genererer feilmeldinger blir gjort om til 'NaT' - Not a Time. 

De andre feltene ser ut til å ha riktig typer. La oss derfor undersøke hvor mye data som mangler i hver kolonne. Denne funksjonen for å oppsummere antall missing i en dataframe har jeg hentet [herfra](https://github.com/WillKoehrsen/machine-learning-project-walkthrough/blob/master/Machine%20Learning%20Project%20Part%201.ipynb). Det prosjektet er vel verdt å sjekke ut!

In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * mis_val / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

missing_values_table(jobs)

Kolonnene som mangler mest data er virksomhet_organisasjonsnr og virksomhet_navn, med i underkant av 20% missing. Resten av kolonnene mangler lite data, og 20% er heller ikke veldig mye. Det er veldig bra! La oss gå videre med å utforske datasettet litt nøyere i neste notebook.